In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [6]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [16]:
html=requests.get(static_url)
falcon9_html = html.text

In [20]:
soup = BeautifulSoup(falcon9_html, 'html.parser')

In [22]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [56]:
html_tables=soup.find_all(name='tr')

In [58]:
first_launch_table = html_tables[2]
print(first_launch_table)

<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>


In [61]:
column_names = []

# Find all 'th' elements in the first launch table
header_cells = first_launch_table.find_all(name='th')

# Iterate over each 'th' element
for header in header_cells:
    # Extract column name using the provided function
    column_name = extract_column_from_header(header)
    
    # Append to column_names if the name is non-empty
    if column_name is not None and len(column_name) > 0:
        column_names.append(column_name)

In [63]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [121]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [123]:
import re

# Function to clean the text by removing references like [8], [e], etc.
def clean_text(text):
    # Ensure text is not None before processing
    if text is None:
        return "Unknown"  # Or return an empty string `""` if you prefer
    
    # Remove reference links or footnotes, e.g., [8], [e], etc.
    text = re.sub(r'\[.*?\]', '', text)
    return text.strip()  # Remove leading and trailing whitespace

# Function to handle missing values
def handle_missing_value(value):
    # If the value is a string, check for common missing value patterns
    if isinstance(value, str):
        if value in ['N/A', '—', 'N/A [e]'] or '[e]' in value:
            return None  # Replace missing values with None or a placeholder
        return value.strip()  # Otherwise, return the cleaned string

    # If the value is numeric, return it as is
    return value


# Function to clean formatting issues
def clean_formatting(text):
    # Remove extra spaces, fix punctuation, etc.
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with one
    return text.strip()  # Remove leading/trailing whitespace

In [124]:
launch_dict_list=[]
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict_row = dict.fromkeys(launch_dict.keys())
            launch_dict_row['Flight No.'] = flight_number
            print(flight_number)
            
            # Date value
            datatimelist=date_time(row[0])
            
            date = clean_text(datatimelist[0])
            launch_dict_row['Date']= date
            print(date)
            
            # Time value
            time = clean_text(datatimelist[1])
            launch_dict_row['Time']= time
            print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string if row[1].a else None
                
            launch_dict_row['Version Booster']=handle_missing_value(bv)
            print(bv)
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict_row['Launch site']=launch_site
            print(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict_row['Payload']=clean_text(payload)
            print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict_row['Payload mass']=handle_missing_value(payload_mass)
            print(payload)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict_row['Orbit']= handle_missing_value(orbit)
            print(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict_row['Customer']=clean_text(customer)
            print(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict_row['Launch outcome']= clean_text(launch_outcome)
            print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict_row['Booster landing']= booster_landing
            print(booster_landing)


            launch_dict_list.append(launch_dict_row)



1
4 June 2010,
18:45
F9 v1.07B0003.18
CCAFS
Dragon Spacecraft Qualification Unit
Dragon Spacecraft Qualification Unit
LEO
SpaceX
Success

Failure
2
8 December 2010,
15:43
F9 v1.07B0004.18
CCAFS
Dragon
Dragon
LEO
NASA
Success
Failure
3
22 May 2012,
07:44
F9 v1.07B0005.18
CCAFS
Dragon
Dragon
LEO
NASA
Success
No attempt

4
8 October 2012,
00:35
F9 v1.07B0006.18
CCAFS
SpaceX CRS-1
SpaceX CRS-1
LEO
NASA
Success

No attempt
5
1 March 2013,
15:10
F9 v1.07B0007.18
CCAFS
SpaceX CRS-2
SpaceX CRS-2
LEO
NASA
Success

No attempt

6
29 September 2013,
16:00
F9 v1.17B10038
VAFB
CASSIOPE
CASSIOPE
Polar orbit
MDA
Success
Uncontrolled
7
3 December 2013,
22:41
F9 v1.1
CCAFS
SES-8
SES-8
GTO
SES
Success
No attempt
8
6 January 2014,
22:06
F9 v1.1
CCAFS
Thaicom 6
Thaicom 6
GTO
Thaicom
Success
No attempt
9
18 April 2014,
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
SpaceX CRS-3
LEO
NASA
Success

Controlled
10
14 July 2014,
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
Orbcomm-OG2
LEO
Orbcomm
Success
Controlled
11
5 A

In [127]:
df= pd.DataFrame(launch_dict_list)

In [129]:
df.head(20)

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.07B0003.18,Failure,"4 June 2010,",18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,"8 December 2010,",15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,"22 May 2012,",07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.07B0006.18,No attempt,"8 October 2012,",00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.07B0007.18,No attempt\n,"1 March 2013,",15:10
5,6,VAFB,CASSIOPE,500 kg,Polar orbit,MDA,Success,F9 v1.17B10038,Uncontrolled,"29 September 2013,",16:00
6,7,CCAFS,SES-8,"3,170 kg",GTO,SES,Success,F9 v1.1,No attempt,"3 December 2013,",22:41
7,8,CCAFS,Thaicom 6,"3,325 kg",GTO,Thaicom,Success,F9 v1.1,No attempt,"6 January 2014,",22:06
8,9,Cape Canaveral,SpaceX CRS-3,"2,296 kg",LEO,NASA,Success,F9 v1.1,Controlled,"18 April 2014,",19:25
9,10,Cape Canaveral,Orbcomm-OG2,"1,316 kg",LEO,Orbcomm,Success,F9 v1.1,Controlled,"14 July 2014,",15:15


In [131]:
df.to_csv('spacex_web_scraped.csv', index=False)